In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [12]:
%load_ext autoreload
%autoreload 2

from os.path import join

import auxiliary as auxi
from cleanData import CleanData
from splitData import SplitData
from itemBasedSim import ItemBasedSim

from pyspark.sql import SQLContext

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
path_root = "/home/jovyan/work/data"
path_pickle_train = join(path_root, "cache/two_domain/split_data/train")
path_pickle_test = join(path_root, "cache/two_domain/split_data/test")

In [4]:
trainRDD = sc.pickleFile(path_pickle_train)
trainRDD = sc.parallelize(trainRDD.take(1000))
testRDD = sc.pickleFile(path_pickle_test)

In [5]:
itembasedsim = ItemBasedSim(method='cosine', num_atleast=50)
universal_user_info = itembasedsim.get_universal_user_info(trainRDD)
user_info = sc.broadcast(universal_user_info.collectAsMap())

universal_item_info = itembasedsim.get_universal_item_info(trainRDD, user_info)
item_info = sc.broadcast(universal_item_info.collectAsMap())

In [13]:
item2item_simRDD = itembasedsim.calculate_item2item_sim(trainRDD, item_info, user_info).cache()

In [14]:
sqlContext                      = SQLContext(sc)

item2item_simDF = itembasedsim.build_sim_DF(item2item_simRDD)

item2item_simDF.registerTempTable("sim_table")
BB_item_list = sqlContext.sql(
    "SELECT DISTINCT id1 FROM sim_table WHERE label = 1").map(
    lambda line: line.id1).collect()
BB_item_bd = sc.broadcast(BB_item_list)

itemSimRDD = itembasedsim.get_item_sim(item2item_simRDD)